Código para cliente y servidor iterativo no bloqueante con FCTLN (no usando select()) escrito en lenguaje C usando sockets.

Código en lenguaje C para un cliente y servidor iterativo TCP no bloqueante usando FCTLN (fcntl()) para configurar los sockets como no bloqueantes, sin usar select() ni poll().


Servidor no bloqueante (iterativo) TCP:

In [1]:
echo '// servidor.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <fcntl.h>
#include <errno.h>
#include <netinet/in.h>
#include <sys/socket.h>

#define PUERTO 8080
#define BUFFER_SIZE 1024

int hacer_no_bloqueante(int fd) {
    int flags = fcntl(fd, F_GETFL, 0);
    return fcntl(fd, F_SETFL, flags | O_NONBLOCK);
}

int main() {
    int servidor_fd, cliente_fd;
    struct sockaddr_in direccion;
    socklen_t addrlen = sizeof(direccion);
    char buffer[BUFFER_SIZE];

    servidor_fd = socket(AF_INET, SOCK_STREAM, 0);
    if (servidor_fd == -1) {
        perror("socket");
        exit(EXIT_FAILURE);
    }

    hacer_no_bloqueante(servidor_fd);

    direccion.sin_family = AF_INET;
    direccion.sin_addr.s_addr = INADDR_ANY;
    direccion.sin_port = htons(PUERTO);

    if (bind(servidor_fd, (struct sockaddr*)&direccion, sizeof(direccion)) < 0) {
        perror("bind");
        exit(EXIT_FAILURE);
    }

    if (listen(servidor_fd, 5) < 0) {
        perror("listen");
        exit(EXIT_FAILURE);
    }

    printf("Servidor escuchando en puerto %d\n", PUERTO);

    while (1) {
        cliente_fd = accept(servidor_fd, (struct sockaddr*)&direccion, &addrlen);
        if (cliente_fd < 0) {
            if (errno == EWOULDBLOCK || errno == EAGAIN) {
                usleep(100000);  // Espera breve para evitar uso excesivo de CPU
                continue;
            } else {
                perror("accept");
                break;
            }
        }

        hacer_no_bloqueante(cliente_fd);

        printf("Cliente conectado\n");

        while (1) {
            ssize_t leidos = read(cliente_fd, buffer, BUFFER_SIZE - 1);
            if (leidos < 0) {
                if (errno == EWOULDBLOCK || errno == EAGAIN) {
                    usleep(100000);
                    continue;
                } else {
                    perror("read");
                    break;
                }
            } else if (leidos == 0) {
                printf("Cliente desconectado\n");
                break;
            }

            buffer[leidos] = '\0';
            printf("Cliente dice: %s", buffer);

            char respuesta[] = "Mensaje recibido\n";
            write(cliente_fd, respuesta, strlen(respuesta));
        }

        close(cliente_fd);
    }

    close(servidor_fd);
    return 0;
}' > servidor_no_bloqueante_iterativo_TCP.c

In [4]:
gcc servidor_no_bloqueante_iterativo_TCP.c -o servidor_no_bloqueante_iterativo_TCP | ls

 cliente
 cliente.c
 cliente_no_bloqueante2
 cliente_no_bloqueante2.c
 Cliente_Servidor_No_Bloqueante_C.ipynb
'Cliente Servidor TCP con bash.ipynb'
 cliente_tcp_bash
 cliente_tcp_bash.c
 cliente_tcp_c.ipynb
'prueba de C con bash.ipynb'
'prueba de C.ipynb'
 servidor
 servidor.c
 servidor_no_bloqueante2
 servidor_no_bloqueante2.c
 servidor_no_bloqueante_iterativo_TCP
 servidor_no_bloqueante_iterativo_TCP.c
 servidor_tcp_bash
 servidor_tcp_bash.c
 servidor_tcp_c.ipynb
 servidor_tcp_concurrente_bash
 servidor_tcp_concurrente_bash.c
 TP1_Calzetta.ipynb
 TP1_Cliente_Servidor_R1.ipynb
 TP1_Cliente_Servidor_R2.ipynb


In [5]:
gnome-terminal -- bash -c "./servidor_no_bloqueante_iterativo_TCP; echo 'Presionar ENTER para cerrar...'; read"

Cliente no bloqueante TCP:

In [6]:
echo '// cliente.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <fcntl.h>
#include <errno.h>
#include <arpa/inet.h>
#include <netinet/in.h>

#define PUERTO 8080
#define BUFFER_SIZE 1024

int hacer_no_bloqueante(int fd) {
    int flags = fcntl(fd, F_GETFL, 0);
    return fcntl(fd, F_SETFL, flags | O_NONBLOCK);
}

int main() {
    int sock;
    struct sockaddr_in servidor;
    char buffer[BUFFER_SIZE];

    sock = socket(AF_INET, SOCK_STREAM, 0);
    if (sock < 0) {
        perror("socket");
        exit(EXIT_FAILURE);
    }

    hacer_no_bloqueante(sock);

    servidor.sin_family = AF_INET;
    servidor.sin_port = htons(PUERTO);
    servidor.sin_addr.s_addr = inet_addr("127.0.0.1");

    int res = connect(sock, (struct sockaddr*)&servidor, sizeof(servidor));
    if (res < 0 && errno != EINPROGRESS) {
        perror("connect");
        close(sock);
        exit(EXIT_FAILURE);
    }

    printf("Conectado al servidor\n");

    while (1) {
        printf("Escribe mensaje: ");
        fgets(buffer, BUFFER_SIZE, stdin);

        ssize_t enviados = write(sock, buffer, strlen(buffer));
        if (enviados < 0) {
            perror("write");
            continue;
        }

        usleep(100000); // Esperar antes de intentar leer

        ssize_t recibidos = read(sock, buffer, BUFFER_SIZE - 1);
        if (recibidos > 0) {
            buffer[recibidos] = '\0';
            printf("Respuesta del servidor: %s\n", buffer);
        }
    }

    close(sock);
    return 0;
}' > cliente_no_bloqueante_iterativo_TCP.c

In [7]:
gcc cliente_no_bloqueante_iterativo_TCP.c -o cliente_no_bloqueante_iterativo_TCP | ls

 cliente
 cliente.c
 cliente_no_bloqueante2
 cliente_no_bloqueante2.c
 cliente_no_bloqueante_iterativo_TCP.c
 Cliente_Servidor_No_Bloqueante_C.ipynb
'Cliente Servidor TCP con bash.ipynb'
 cliente_tcp_bash
 cliente_tcp_bash.c
 cliente_tcp_c.ipynb
'prueba de C con bash.ipynb'
'prueba de C.ipynb'
 servidor
 servidor.c
 servidor_no_bloqueante2
 servidor_no_bloqueante2.c
 servidor_no_bloqueante_iterativo_TCP
 servidor_no_bloqueante_iterativo_TCP.c
 servidor_tcp_bash
 servidor_tcp_bash.c
 servidor_tcp_c.ipynb
 servidor_tcp_concurrente_bash
 servidor_tcp_concurrente_bash.c
 TP1_Calzetta.ipynb
 TP1_Cliente_Servidor_R1.ipynb
 TP1_Cliente_Servidor_R2.ipynb


In [8]:
gnome-terminal -- bash -c "./cliente_no_bloqueante_iterativo_TCP; echo 'Presionar ENTER para cerrar...'; read"

Código en lenguaje C para un cliente y servidor iterativo UDP no bloqueante usando FCTLN (fcntl()) para configurar los sockets como no bloqueantes, sin usar select() ni poll().

Servidor no bloqueante (iterativo) UDP:

In [13]:
echo '// servidor_udp.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <fcntl.h>
#include <errno.h>
#include <netinet/in.h>
#include <sys/socket.h>

#define PUERTO 8081
#define BUFFER_SIZE 1024

int hacer_no_bloqueante(int fd) {
    int flags = fcntl(fd, F_GETFL, 0);
    return fcntl(fd, F_SETFL, flags | O_NONBLOCK);
}

int main() {
    int sockfd;
    struct sockaddr_in servidor, cliente;
    socklen_t cliente_len = sizeof(cliente);
    char buffer[BUFFER_SIZE];

    sockfd = socket(AF_INET, SOCK_DGRAM, 0);
    if (sockfd < 0) {
        perror("socket");
        exit(EXIT_FAILURE);
    }

    hacer_no_bloqueante(sockfd);

    memset(&servidor, 0, sizeof(servidor));
    servidor.sin_family = AF_INET;
    servidor.sin_addr.s_addr = INADDR_ANY;
    servidor.sin_port = htons(PUERTO);

    if (bind(sockfd, (struct sockaddr*)&servidor, sizeof(servidor)) < 0) {
        perror("bind");
        close(sockfd);
        exit(EXIT_FAILURE);
    }

    printf("Servidor UDP no bloqueante escuchando en puerto %d...\n", PUERTO);

    while (1) {
        ssize_t n = recvfrom(sockfd, buffer, BUFFER_SIZE - 1, 0,
                             (struct sockaddr*)&cliente, &cliente_len);
        if (n < 0) {
            if (errno == EWOULDBLOCK || errno == EAGAIN) {
                usleep(100000); // Espera breve
                continue;
            } else {
                perror("recvfrom");
                break;
            }
        }

        buffer[n] = '\0';
        printf("Recibido de cliente: %s\n", buffer);

        const char* respuesta = "Mensaje recibido por el servidor UDP\n";
        sendto(sockfd, respuesta, strlen(respuesta), 0,
               (struct sockaddr*)&cliente, cliente_len);
    }

    close(sockfd);
    return 0;
}' > servidor_no_bloqueante_iterativo_UDP.c

In [16]:
gcc servidor_no_bloqueante_iterativo_UDP.c -o servidor_no_bloqueante_iterativo_UDP | ls

 cliente
 cliente.c
 cliente_no_bloqueante2
 cliente_no_bloqueante2.c
 cliente_no_bloqueante_iterativo_TCP
 cliente_no_bloqueante_iterativo_TCP.c
 Cliente_Servidor_No_Bloqueante_C.ipynb
'Cliente Servidor TCP con bash.ipynb'
 cliente_tcp_bash
 cliente_tcp_bash.c
 cliente_tcp_c.ipynb
'prueba de C con bash.ipynb'
'prueba de C.ipynb'
 servidor
 servidor.c
 servidor_no_bloqueante2
 servidor_no_bloqueante2.c
 servidor_no_bloqueante_iterativo_TCP
 servidor_no_bloqueante_iterativo_TCP.c
 servidor_no_bloqueante_iterativo_UDP
 servidor_no_bloqueante_iterativo_UDP.c
 servidor_tcp_bash
 servidor_tcp_bash.c
 servidor_tcp_c.ipynb
 servidor_tcp_concurrente_bash
 servidor_tcp_concurrente_bash.c
 TP1_Calzetta.ipynb
 TP1_Cliente_Servidor_R1.ipynb
 TP1_Cliente_Servidor_R2.ipynb


In [17]:
gnome-terminal -- bash -c "./servidor_no_bloqueante_iterativo_UDP; echo 'Presionar ENTER para cerrar...'; read"

Cliente no bloqueante (iterativo) UDP:

In [18]:
echo '// cliente_udp.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <fcntl.h>
#include <errno.h>
#include <arpa/inet.h>
#include <netinet/in.h>

#define PUERTO 8081
#define BUFFER_SIZE 1024

int hacer_no_bloqueante(int fd) {
    int flags = fcntl(fd, F_GETFL, 0);
    return fcntl(fd, F_SETFL, flags | O_NONBLOCK);
}

int main() {
    int sockfd;
    struct sockaddr_in servidor;
    socklen_t servidor_len = sizeof(servidor);
    char buffer[BUFFER_SIZE];

    sockfd = socket(AF_INET, SOCK_DGRAM, 0);
    if (sockfd < 0) {
        perror("socket");
        exit(EXIT_FAILURE);
    }

    hacer_no_bloqueante(sockfd);

    memset(&servidor, 0, sizeof(servidor));
    servidor.sin_family = AF_INET;
    servidor.sin_port = htons(PUERTO);
    servidor.sin_addr.s_addr = inet_addr("127.0.0.1");

    printf("Cliente UDP no bloqueante listo para enviar mensajes.\n");

    while (1) {
        printf("Escribe mensaje: ");
        fgets(buffer, BUFFER_SIZE, stdin);

        sendto(sockfd, buffer, strlen(buffer), 0,
               (struct sockaddr*)&servidor, servidor_len);

        usleep(100000); // Espera antes de intentar leer

        ssize_t n = recvfrom(sockfd, buffer, BUFFER_SIZE - 1, 0,
                             NULL, NULL);
        if (n > 0) {
            buffer[n] = '\0';
            printf("Respuesta del servidor: %s\n", buffer);
        }
    }

    close(sockfd);
    return 0;
}' > cliente_no_bloqueante_iterativo_UDP.c

In [19]:
gcc cliente_no_bloqueante_iterativo_UDP.c -o cliente_no_bloqueante_iterativo_UDP | ls

 cliente
 cliente.c
 cliente_no_bloqueante2
 cliente_no_bloqueante2.c
 cliente_no_bloqueante_iterativo_TCP
 cliente_no_bloqueante_iterativo_TCP.c
 cliente_no_bloqueante_iterativo_UDP.c
 Cliente_Servidor_No_Bloqueante_C.ipynb
'Cliente Servidor TCP con bash.ipynb'
 cliente_tcp_bash
 cliente_tcp_bash.c
 cliente_tcp_c.ipynb
'prueba de C con bash.ipynb'
'prueba de C.ipynb'
 servidor
 servidor.c
 servidor_no_bloqueante2
 servidor_no_bloqueante2.c
 servidor_no_bloqueante_iterativo_TCP
 servidor_no_bloqueante_iterativo_TCP.c
 servidor_no_bloqueante_iterativo_UDP
 servidor_no_bloqueante_iterativo_UDP.c
 servidor_tcp_bash
 servidor_tcp_bash.c
 servidor_tcp_c.ipynb
 servidor_tcp_concurrente_bash
 servidor_tcp_concurrente_bash.c
 TP1_Calzetta.ipynb
 TP1_Cliente_Servidor_R1.ipynb
 TP1_Cliente_Servidor_R2.ipynb


In [20]:
gnome-terminal -- bash -c "./cliente_no_bloqueante_iterativo_UDP; echo 'Presionar ENTER para cerrar...'; read"